In [24]:
import os
import time
from rich.pretty import pprint
from typing import List, Dict, Any, Tuple
from pycomex.functional.experiment import Experiment


PATH: str = os.getcwd()
RESULTS_PATH: str = os.path.join(PATH, 'results')

IDENTIFIER = 'ex_02_b'

In [25]:
# ~ loading the experiments

time_start = time.time()
experiments: List[Experiment] = []
for root, folders, files in os.walk(RESULTS_PATH):
    
    for folder in folders:
        folder_path = os.path.join(root, folder)
        
        experiment_path = os.path.join(folder_path, 'experiment_code.py')
        if not os.path.exists(experiment_path):
            continue
        try:
            experiment = Experiment.load(experiment_path)
        except:
            continue
        
        if 'IDENTIFIER' not in experiment.parameters or experiment.parameters['IDENTIFIER'] != IDENTIFIER:
            continue
        
        experiments.append(experiment)
        
time_end = time.time()
duration = time_end - time_start
print(f'loaded {len(experiments)} in {duration:.2f} seconds')

example_experiment = experiments[0]
print('example experiment:')
pprint(example_experiment.data, max_length=5)

if 'r2' in example_experiment.data:
    performance_key = 'r2'
    performance_name = r'$R^2 \uparrow$'
elif 'accuracy' in example_experiment.data:
    performance_key = 'accuracy'
    performance_name = r'$\text{Accuracy} \uparrow$'
    
print(f'performance key: {performance_key}')

loaded 14 in 2.27 seconds
example experiment:


{
│   'dataset_path': '/media/ssd/.visual_graph_datasets/datasets/aqsoldb',
│   'node_dim': 45,
│   'edge_dim': 11,
│   'output_dim': 1,
│   'indices': {
│   │   'train': [0, 1, 2, 3, 5, ... +7437],
│   │   'val': [1016, 1656, 1537, 6605, 3084, ... +821],
│   │   'test': [1831, 2348, 2249, 6321, 3491, ... +995],
│   │   'example': [859, 1076, 7170, 6330, 7839],
│   │   'train_filtered': [0, 1, 2, 3, 5, ... +7437]
│   },
│   ... +14
}

performance key: r2


In [26]:
from collections import defaultdict

# ~ constructing experiment map

# The keys will be a combination of (learning_rate, lr_scheduler) experiment parameters
# the values will be a list of experiments with the same parameters but different keys.
key_experiments_map: Dict[Tuple[str, str], List[Experiment]] = defaultdict(list)
experiments_all = experiments

for experiment in experiments:
    
    batch_size = experiment.parameters['BATCH_SIZE']
    
    key = batch_size
    key_experiments_map[key].append(experiment)
    
pprint(key_experiments_map, max_length=5)

defaultdict(<class 'list'>, {
│   48: [
│   │   <pycomex.functional.experiment.Experiment object at 0x79866dcab910>,
│   │   <pycomex.functional.experiment.Experiment object at 0x79864ee75330>
│   ],
│   128: [
│   │   <pycomex.functional.experiment.Experiment object at 0x79865a442a40>,
│   │   <pycomex.functional.experiment.Experiment object at 0x79864ea23580>
│   ],
│   16: [
│   │   <pycomex.functional.experiment.Experiment object at 0x79864eb1ff70>,
│   │   <pycomex.functional.experiment.Experiment object at 0x79864eb1d570>,
│   │   <pycomex.functional.experiment.Experiment object at 0x7986580f7bb0>
│   ],
│   64: [
│   │   <pycomex.functional.experiment.Experiment object at 0x79864ea21540>,
│   │   <pycomex.functional.experiment.Experiment object at 0x79866cffe7a0>
│   ],
│   96: [
│   │   <pycomex.functional.experiment.Experiment object at 0x79864e8c8eb0>,
│   │   <pycomex.functional.experiment.Experiment object at 0x7986457034c0>
│   ],
│   ... +1
})

In [27]:
import numpy as np

# ~ extracting results from experiments

hist_len = 5

column_names = [
    'Batch Size', 
    performance_name,
    r'$\text{Expl. Acc.} \uparrow$',
    r'$\mathcal{L}_{pred} \downarrow$',
    r'$\mathcal{L}_{sparse} \downarrow$',
    r'$\mathcal{L}_{fid} \downarrow$',
]

rows: List[list] = []

for batch_size, experiments in key_experiments_map.items():
    
    row = [str(batch_size)]
    
    row.append([np.mean(exp.data[performance_key][-hist_len:]) for exp in experiments])
    row.append([np.mean(exp.data['approx'][-hist_len:]) for exp in experiments])
    row.append([np.mean(exp.data['loss_pred'][-hist_len:]) for exp in experiments])
    row.append([np.mean(exp.data['loss_spar'][-hist_len:]) for exp in experiments])
    row.append([np.mean(exp.data['loss_fid'][-hist_len:]) for exp in experiments])
        
    rows.append(row)
    
rows.sort(key=lambda x: (float(x[0]), ))
    
pprint(rows, max_length=5)

[
│   [
│   │   '16',
│   │   [0.8159101981792324, 0.8117936597843372, 0.7681723244576805],
│   │   [0.7652542372881357, 0.7749394673123486, 0.7378934624697336],
│   │   [0.20354368388652802, 0.2023317337036133, 0.19226012825965882],
│   │   [0.33663763403892516, 0.33798574209213256, 0.34289935827255247],
│   │   ... +1
│   ],
│   [
│   │   '32',
│   │   [0.8698898614325603, 0.8421457079596231, 0.8565907549807111],
│   │   [0.8312348668280872, 0.8187651331719128, 0.8089588377723972],
│   │   [0.1992706686258316, 0.18811048567295074, 0.17750626802444458],
│   │   [0.35045352578163147, 0.3299020051956177, 0.3523350298404694],
│   │   ... +1
│   ],
│   [
│   │   '48',
│   │   [0.7538407478590388, 0.7563858306120359],
│   │   [0.7641646489104116, 0.7790556900726393],
│   │   [0.22070904672145844, 0.20213216841220855],
│   │   [0.3581511855125427, 0.3380286514759064],
│   │   ... +1
│   ],
│   [
│   │   '64',
│   │   [0.8516366737345858, 0.8662204011134959],
│   │   [0.8180387409200968, 0.797094430992736],
│   │   [0.15964336693286896, 0.19157985746860504],
│   │   [0.31943429112434385, 0.38464451432228086],
│   │   ... +1
│   ],
│   [
│   │   '96',
│   │   [0.8338389111591556, 0.8804693845863394],
│   │   [0.8035108958837771, 0.8154963680387409],
│   │   [0.21070356369018556, 0.2014454871416092],
│   │   [0.3396181583404541, 0.3559178948402405],
│   │   ... +1
│   ],
│   ... +1
]

In [28]:
from graph_attention_student.utils import render_latex
from graph_attention_student.utils import latex_table

dataset = example_experiment.parameters['VISUAL_GRAPH_DATASET'].replace('_', ' ')

# ~ rendering latex

tex_content, tex_table = latex_table(
    column_names=column_names,
    rows=rows,
    caption=f'learning rate comparison for "{dataset}"'
)
print(tex_table)

tex_path = os.path.join(PATH, '_results_ex_02.tex')
# with open(tex_path, 'w') as file:
#     file.write(tex_table)
#     pass

pdf_path = os.path.join(PATH, '_results_ex_02.pdf')
render_latex({'content': tex_table}, pdf_path)

\begin{tabular}{ cccccc }
% The caption text
\multicolumn{ 6 }{c}{ learning rate comparison for "aqsoldb" } \\[0.2cm]
%\vspace*{0.2cm}
% -- table header --
\toprule
Batch Size &
$R^2 \uparrow$ &
$\text{Expl. Acc.} \uparrow$ &
$\mathcal{L}_{pred} \downarrow$ &
$\mathcal{L}_{sparse} \downarrow$ &
$\mathcal{L}_{fid} \downarrow$ \\

\midrule
% -- table content --
% row 1
16 &
$0.80 {\color{gray} \pm \mathsmaller{ 0.02 } }$ &
$0.76 {\color{gray} \pm \mathsmaller{ 0.02 } }$ &
$0.20 {\color{gray} \pm \mathsmaller{ 0.01 } }$ &
$0.34 {\color{gray} \pm \mathsmaller{ 0.00 } }$ &
$0.25 {\color{gray} \pm \mathsmaller{ 0.02 } }$ 
\\
% row 2
32 &
$0.86 {\color{gray} \pm \mathsmaller{ 0.01 } }$ &
$0.82 {\color{gray} \pm \mathsmaller{ 0.01 } }$ &
$0.19 {\color{gray} \pm \mathsmaller{ 0.01 } }$ &
$0.34 {\color{gray} \pm \mathsmaller{ 0.01 } }$ &
$0.25 {\color{gray} \pm \mathsmaller{ 0.03 } }$ 
\\
% row 3
48 &
$0.76 {\color{gray} \pm \mathsmaller{ 0.00 } }$ &
$0.77 {\color{gray} \pm \mathsmaller{ 0.01 } 